# TensorRT EfficientDet-Lite Model Conversion AutoML Models to ONNX Model

This notebook contains a sample that converts EfficientDet-Lite's AutoML Model into an ONNX model for running on TensorRT.  

Reference
- [EfficientDet Object Detection in TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/efficientdet)
- [EfficientDet](https://github.com/google/automl/tree/master/efficientdet)

# Export Saved Model

## Clone [google/automl](https://github.com/google/automl) repository and install dependency.


In [ ]:
%%bash

cd /content
git clone https://github.com/google/automl
cd automl
git checkout 38ecb93913fc19e429ab2a572f1aa8f5286723cf
cd efficientdet
pip3 install -r requirements.txt

In [ ]:
import os
import yaml

os.environ['PYTHONPATH'] = '/content/automl/efficientdet:' + os.environ['PYTHONPATH']
print(os.environ['PYTHONPATH'])

## Download EfficentDet Lite checkpoint and export saved model.

### Download checkpoint

Select the checkpoint you want to export.

In [ ]:
#@title Select EfficientDet-lite model.

checkpoints = 'efficientdet-lite1' #@param ["efficientdet-lite0", "efficientdet-lite1", "efficientdet-lite2", "efficientdet-lite3", "efficientdet-lite3x", "efficientdet-lite4"] {allow-input: false}


In [ ]:
file_name = checkpoints + ".tgz"
path = "https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/" + file_name

!wget $path
!tar xf $file_name

In [ ]:
size = {
    "efficientdet-lite0":"320x320",
    "efficientdet-lite1":"384x384",
    "efficientdet-lite2":"448x448",
    "efficientdet-lite3":"512x512",
    "efficientdet-lite3x":"640x640",
    "efficientdet-lite4":"640x640",
}

### Set NMS configs

In [ ]:
obj = { 'image_size': size[checkpoints],
       'nms_configs': {
           'method': 'hard',
           'iou_thresh': 0.35,
           'score_thresh': 0.,
           'sigma': 0.0,
           'pyfunc': False,
           'max_nms_inputs': 0,
           'max_output_size': 100
           }
       }

with open('saved_model.yaml', 'w') as file:
    yaml.dump(obj, file)

In [ ]:
!cat saved_model.yaml

### Export Saved Model

In [ ]:
model_dir = os.path.join("/content", checkpoints)
saved_model_dir = os.path.join("/content", "saved_model_" + checkpoints)

In [ ]:
# Export Saved model
!python /content/automl/efficientdet/tf2/inspector.py \
    --mode=export \
    --model_name=$checkpoints \
    --model_dir=$model_dir \
    --saved_model_dir=$saved_model_dir \
    --hparams=/content/saved_model.yaml

# Export ONNX

## Clone [NVIDIA/TensorRT](https://github.com/NVIDIA/TensorRT) repository and install dependency.

In [ ]:
%%bash

cd /content
git clone https://github.com/NVIDIA/TensorRT
cd TensorRT
git checkout 2d517d270e3697a5775e7861873dc21d5fba6bae
cd /content/TensorRT/samples/python/efficientdet

pip3 install -r requirements.txt
pip3 install onnx-graphsurgeon --index-url https://pypi.ngc.nvidia.com

In [ ]:
%cd /content/TensorRT/samples/python/efficientdet

## Export ONNX Model

In [ ]:
input_shape = {
    "efficientdet-lite0":"1,320,320,3",
    "efficientdet-lite1":"1,384,384,3",
    "efficientdet-lite2":"1,448,448,3",
    "efficientdet-lite3":"1,512,512,3",
    "efficientdet-lite3x":"1,640,640,3",
    "efficientdet-lite4":"1,640,640,3",
}

In [ ]:
input = input_shape[checkpoints]
output = os.path.join("/content", checkpoints + ".onnx")

In [ ]:
!python3 create_onnx.py \
    --input_shape $input \
    --saved_model $saved_model_dir \
    --onnx $output

Now Download ONNX Model.